In [13]:
"""
Authored By: Buz Galbraith

"""

import pandas as pd
import json

class StateTraceParser:
    def __init__(self, path):
        """ 1. data is just the json dictionary
            2. position_time is a dictionary with key values for each time step related to the bals position. 
            3. velocity_time dictionary with time keys and velcoity values 
            4.object_position_time dictionary with time keys and object position values
            5.note_times dictionary with time keys and binary variable for if a note is posted
            6.reset_times dictionary with time keys and binary variable for if a reset is made
            7.lastStepNum the final time step 
            8. walls a nested dict with keys X and Y and sub keys max and min representing the game boundry
            9. df time indexed aggreagate dictioanry . 
            10. wall_hits subset of df where the ball hits walls. 
            11. not_wall_hits subset of df where teh ball does not hit the wall 9is the complement of wall hits)
        """
        self.data = json.load(open(path))
        self.tags = self.data["foundObjectsTags"]
        self.dim = len(self.tags)
        self.lastStepNum = self.data["lastStepNum"]
        self.walls = {"X": {"Max": self.data["boxMaxX"],
                            "Min": self.data["boxMinX"]},
                      "Y": {"Max": self.data["boxMaxY"],
                            "Min": self.data["boxMinY"]}}
        self.make_df()

        self.wall_hits = self.df[[self.row_check(
            i) for i in range(len(self.df["ball_x"]))]]
        self.not_wall_hits = self.df[[self.row_check(
            i) == False for i in range(len(self.df["ball_x"]))]]
        self.above_bucket = self.df[self.df["ball_y"] >= self.df["bucket_y"]]
        self.above_corner = self.df[self.df["ball_y"] >= self.df["corner_y"]]
        self.above_triangle = self.df[self.df["ball_y"]
                                      >= self.df["triangle_y"]]
        self.above_gear = self.df[self.df["ball_y"] >= self.df["gear_y"]]
        self.above_crate = self.df[self.df["ball_y"] >= self.df["crate_y"]]

    def make_df(self):
        def makeDict(posVec):
            dct = {}
            for i in range(self.dim):
                dct.update({f"{self.tags[i]}_x": posVec[i]["x"],
                            f"{self.tags[i]}_y": posVec[i]["y"]})
            return dct
        reshaped = [makeDict(self.data["objectPositions"][i:i+self.dim]) for i
                    in range(0, len(self.data["objectPositions"]), self.dim)]
        objPos = pd.DataFrame(reshaped)
        ballPos = pd.DataFrame(self.data["ballPositions"])[["x", "y"]]
        ballPos.columns = ["ball_x", "ball_y"]
        self.df = pd.concat((ballPos, objPos), axis=1)
        if len(self.data["velocitiesCT"])>0:
            self.df.loc[self.data["velocitiesCT"], ["velocity_x", "velocity_y"]] = list(zip(pd.DataFrame(self.data["velocities"])["x"], pd.DataFrame(self.data["velocities"])["y"]))
        else:
            self.df[["velocity_x", "velocity_y"]] = 0.0
        self.df.loc[self.data["resetCT"], "reset"] = 1
        self.df.loc[self.data["notesCT"], "note"] = self.data["notes"]
        self.df["note"].fillna("", inplace=True)
        self.df.fillna(0, inplace=True)

    def row_check(self, x): 
        return (self.df["ball_x"][x] <= self.walls["X"]["Min"]) or \
                (self.df["ball_x"][x] >= self.walls["X"]["Max"]) or \
                (self.df["ball_y"][x] <= self.walls["Y"]["Min"]) or \
                (self.df["ball_y"][x] >= self.walls["Y"]["Max"])

    def ballInBucket(self, timestep):
        MIN_X_DELTA = -0.1927506923675537
        MAX_X_DELTA = 0.2523689270019531
        MIN_Y_DELTA = -0.24334418773651123
        MAX_Y_DELTA = 0.6142134666442871

        x_delta = self.df.loc[timestep, "ball_x"] - self.df.loc[timestep, "bucket_x"]
        y_delta = self.df.loc[timestep, "ball_y"] - self.df.loc[timestep, "bucket_y"]

        return (MAX_X_DELTA>=x_delta>=MIN_X_DELTA) and (MAX_Y_DELTA>=y_delta>=MIN_Y_DELTA)

In [14]:
parser = StateTraceParser("/Users/aditya/Documents/GitHub/game_creation_research/Object Physics Sandbox/Builds/ExperimenterView_21April22.app/Contents/InteractionLogs/logs_2022_04_23_14_37_32_game2_setup.json")

In [15]:
parser.df

,ball_x,ball_y,corner_x,corner_y,bucket_x,bucket_y,triangle_x,triangle_y,gear_x,gear_y,crate_x,crate_y,velocity_x,velocity_y,reset,note
0,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
1,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
2,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
3,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
4,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,5.340000,3.260000,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,
1413,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,
1414,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,
1415,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,


In [18]:
parser.df[parser.df["note"]!=""]

,ball_x,ball_y,corner_x,corner_y,bucket_x,bucket_y,triangle_x,triangle_y,gear_x,gear_y,crate_x,crate_y,velocity_x,velocity_y,reset,note
903,-1.35,-4.12,5.34,-2.65,5.340000,1.850000,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,box pos\n
1343,-1.35,-4.12,5.34,-2.65,-4.847445,-4.238384,5.34,-1.25,5.34,0.24,-1.262854,2.134222,0.0,0.0,0.0,buvket pos\n
